In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from tqdm import tqdm

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
# 구글 드라이브 불러오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 폴더 내 이미지 리스트 불러오기
img_path = '/content/drive/MyDrive/DF/TS_PET/'
img_list = os.listdir(img_path)
img_list = [img for img in img_list if '(1)' not in img]
print(len(img_list))
img_list = [img for img in img_list if '(2)' not in img]
len(img_list)

2531


2524

In [ ]:
# 폴더 내 json 리스트 불러오기
json_path = '/content/drive/MyDrive/DF/TL_PET_0/'
json_list = os.listdir(json_path)
len(json_list)

10034

In [ ]:
print(len(img_list) == len(json_list)) # 실제 데이터에서는 크기 다를 듯
print(len(img_list))

False
2524


In [ ]:
# tensor를 .npy 파일로 저장하는 함수
# data_np 를 경로 '/content/drive/MyDrive/DF/filename.npy'로 저장
def tensor_toFile(data, filename):
  data_np = data.cpu().numpy()
  np.save('/content/drive/MyDrive/DF/전처리 파일/'+filename, data_np)


# .npy 파일 불러와서 tensor로 저장하는 함수
def file_toTensor(filename):
  np_load = np.load('/content/drive/MyDrive/DF/전처리 파일/'+filename+'.npy')
  img_tensor = torch.from_numpy(np_load)
  return img_tensor

In [ ]:
# 이미지 하나씩 불러와서 자르고 tensor로 저장하기
def img_toTensor(img_list, img_path, json_path):
  tf = transforms.ToTensor()
  img_data = []
  img_class = []

  for i in tqdm(img_list):
    img = PIL.Image.open(img_path+i)
    with open(json_path+i[:-3]+'json', 'r') as f:
      json_data = json.load(f)
    bbox = json_data['annotations'][0]['bbox']
    img_crop = transforms.functional.crop(img, bbox[1], bbox[0], bbox[3], bbox[2]) # 이미지 자르기
    img_crop = img_crop.resize((256,256)) # 이미지 사이즈 통일하기
#    img_crop.show()
    img_t = tf(img_crop) # 이미지 -> tensor로 변환
    img_data.append(img_t)
    img_class.append(json_data['annotations'][0]['category_id'])

  img_data = torch.stack(img_data, 0)
  tensor_toFile(img_data, 'img_data')
  img_class = torch.tensor(img_class)
  tensor_toFile(img_class, 'img_class')

  return img_data, img_class

In [ ]:
data, target = img_toTensor(img_list, img_path, json_path)

100%|██████████| 2524/2524 [23:29<00:00,  1.79it/s]


In [ ]:
print(data.size())
data

torch.Size([2524, 3, 256, 256])


tensor([[[[0.0902, 0.0980, 0.0824,  ..., 0.1020, 0.1020, 0.1098],
          [0.0902, 0.0902, 0.0902,  ..., 0.1137, 0.1059, 0.1020],
          [0.0784, 0.0745, 0.0824,  ..., 0.1216, 0.1137, 0.1137],
          ...,
          [0.1020, 0.1059, 0.1059,  ..., 0.0824, 0.0824, 0.0824],
          [0.1020, 0.1059, 0.1059,  ..., 0.0824, 0.0824, 0.0824],
          [0.1020, 0.1098, 0.1059,  ..., 0.0824, 0.0824, 0.0863]],

         [[0.0784, 0.0863, 0.0706,  ..., 0.1216, 0.1216, 0.1294],
          [0.0745, 0.0745, 0.0745,  ..., 0.1333, 0.1255, 0.1216],
          [0.0588, 0.0627, 0.0706,  ..., 0.1412, 0.1333, 0.1333],
          ...,
          [0.1059, 0.1098, 0.1137,  ..., 0.0902, 0.0902, 0.0902],
          [0.1098, 0.1176, 0.1176,  ..., 0.0902, 0.0902, 0.0902],
          [0.1098, 0.1176, 0.1176,  ..., 0.0902, 0.0902, 0.0941]],

         [[0.0667, 0.0745, 0.0549,  ..., 0.1059, 0.1059, 0.1137],
          [0.0627, 0.0627, 0.0627,  ..., 0.1176, 0.1098, 0.1059],
          [0.0471, 0.0510, 0.0588,  ..., 0

In [ ]:
print(target.size())
target

torch.Size([2524])


tensor([1, 1, 1,  ..., 1, 1, 1])

In [ ]:
data_tensor = file_toTensor('img_data')
target_tensor = file_toTensor('img_class')

In [ ]:
print(target_tensor.size())
target_tensor

torch.Size([2524])


tensor([1, 1, 1,  ..., 1, 1, 1])

In [ ]:
# json 파일 형태
with open('/content/drive/MyDrive/DF/라벨링데이터/TL_PET_0/PET_001_1.json', 'r') as f:
  json_data = json.load(f)
json_data

{'images': [{'width': 2048,
   'height': 2048,
   'id': 1,
   'file_name': 'PET_001_1.jpg'}],
 'categories': [{'id': 1, 'name': 'pet'},
  {'id': 2, 'name': 'ps'},
  {'id': 3, 'name': 'pp'},
  {'id': 4, 'name': 'pe'}],
 'annotations': [{'id': 1,
   'image_id': 1,
   'category_id': 1,
   'segmentation': [[474.0,
     671.0,
     462.0,
     687.0,
     455.0,
     703.0,
     445.0,
     719.0,
     430.0,
     733.0,
     416.0,
     747.0,
     407.0,
     763.0,
     393.0,
     779.0,
     393.0,
     795.0,
     406.0,
     812.0,
     421.0,
     827.0,
     437.0,
     838.0,
     453.0,
     851.0,
     470.0,
     861.0,
     486.0,
     869.0,
     502.0,
     880.0,
     518.0,
     895.0,
     534.0,
     909.0,
     550.0,
     920.0,
     567.0,
     930.0,
     583.0,
     942.0,
     599.0,
     957.0,
     615.0,
     973.0,
     631.0,
     988.0,
     648.0,
     1002.0,
     664.0,
     1013.0,
     680.0,
     1026.0,
     696.0,
     1037.0,
     712.0,
     1049.0,